In [1]:

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
#import wfdb
from random import shuffle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
from torch.autograd import Variable

from __future__ import print_function
import torch.optim as optim
import h5py
import time


In [2]:
class myDataset(Dataset):#dataset
    def __init__(self,mix,m_h,sig_num,seq_len,input_size):
        #path
        #  x0:TF     x1:mask of HS    x2：mask of LS
        x0=np.loadtxt(open(mix,"rb"),delimiter=",",skiprows=0)
        tempt=np.zeros([sig_num,seq_len,input_size]) 
        for i in range(sig_num):  #
            for hang in range(seq_len):
                for lie in range(input_size):
                    tempt[i,hang,lie]=x0[hang+i*seq_len,lie]
        x0=tempt.astype('float32')
        
        x1=np.loadtxt(open(m_h,"rb"),delimiter=",",skiprows=0)
        tempt=np.zeros([sig_num,seq_len,input_size]) 
        for i in range(sig_num):  #
            for hang in range(seq_len):
                for lie in range(input_size):
                    tempt[i,hang,lie]=x1[hang+i*seq_len,lie]
        x1=tempt.astype('float32')    
        
        x0=torch.from_numpy(x0);x1=torch.from_numpy(x1);

        x0=Variable(x0,requires_grad=True);x1=Variable(x1,requires_grad=True);
        
        x0=x0[:,np.newaxis,:,:];x1=x1[:,np.newaxis,:,:];
        x0.cuda(0);x1.cuda(0);

        self.len = x0.shape[0] # shape
        self.x0_data=x0;self.x1_data=x1;
        
   
    def __getitem__(self, index):    
        #getitem是一个magic方法，可以对将来的实例对象执行下标操作,可以通过索引index，将里面的数据拿出来
        return self.x0_data[index].cuda(0),self.x1_data[index].cuda(0)
    
    def __len__(self): #return the number of dataset
        return self.len
      

In [3]:
######################impeordata
batch_train=1          #
seq_len=256         #
input_size=128      #


# dataset=myDataset(mix="train0_mix鼾声_875.csv",m_h="train0_m_h鼾声_875.csv",
#                   sig_num=875,seq_len=seq_len,input_size=input_size)
# train_loader = DataLoader(dataset=dataset, batch_size=batch_train, shuffle=False,num_workers=0) 


dataset=myDataset(mix="mix_60.csv",m_h="m_h_60.csv",
                  sig_num=20,seq_len=seq_len,input_size=input_size)
train_loader = DataLoader(dataset=dataset, batch_size=batch_train, shuffle=False,num_workers=0) 


dataset=myDataset(mix="mix_12.csv",m_h="m_h_12.csv",
                  sig_num=12,seq_len=seq_len,input_size=input_size)
test_loader = DataLoader(dataset=dataset, batch_size=4, shuffle=False,num_workers=0) 


In [4]:
for index, (x0,x1,) in enumerate(train_loader,0): 
    pass
print('*'*50);print(index+1)
print(" x0：mix的尺寸:{},类型:{},梯度:{}".format(x0.shape,x0.dtype,x0.requires_grad))
print(" x1：m_h的尺寸:{},类型:{},梯度:{}".format(x1.shape,x1.dtype,x1.requires_grad))

for index, (x0,x1) in enumerate(test_loader,0): 
    pass
print('*'*50);print(index+1)
print(" x0：mix的尺寸:{},类型:{},梯度:{}".format(x0.shape,x0.dtype,x0.requires_grad))
print(" x1：m_h的尺寸:{},类型:{},梯度:{}".format(x1.shape,x1.dtype,x1.requires_grad))
##[batch,channel,H,W]

print(x0.type())

**************************************************
20
 x0：mix的尺寸:torch.Size([1, 1, 256, 128]),类型:torch.float32,梯度:True
 x1：m_h的尺寸:torch.Size([1, 1, 256, 128]),类型:torch.float32,梯度:True
**************************************************
3
 x0：mix的尺寸:torch.Size([4, 1, 256, 128]),类型:torch.float32,梯度:True
 x1：m_h的尺寸:torch.Size([4, 1, 256, 128]),类型:torch.float32,梯度:True
torch.cuda.FloatTensor


In [10]:
#定义模型
class MyUnet(nn.Module):
    def __init__(self):
        super().__init__()
        #coding（3+1）
        self.bn1= nn.BatchNorm2d(48);self.bn2= nn.BatchNorm2d(96);self.bn3= nn.BatchNorm2d(144);
        self.bn0= nn.BatchNorm2d(1);
        self.encode1_1 =nn.Conv2d(1,48,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode1_2=nn.Conv2d(48,48,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode2_1=nn.Conv2d(48,96,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode2_2=nn.Conv2d(96,96,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode3_1=nn.Conv2d(96,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode3_2=nn.Conv2d(144,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode4_1=nn.Conv2d(144,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.encode4_2=nn.Conv2d(144,144,kernel_size=3,stride=1,padding=1,bias=False)
        
        #decoding（3+1个组）
        self.upsample3= nn.ConvTranspose2d(144,144,kernel_size=2,stride=2,padding=0);                                       
        self.decode3_1 =nn.Conv2d(288,144,kernel_size=3,stride=1,padding=1,bias=False)
        self.decode3_2 =nn.Conv2d(144,144,kernel_size=3,stride=1,padding=1,bias=False)
        
        self.upsample2= nn.ConvTranspose2d(144,144,kernel_size=2,stride=2,padding=0);
        self.decode2_1 =nn.Conv2d(240,96,kernel_size=3,stride=1,padding=1,bias=False)
        self.decode2_2 =nn.Conv2d(96,96,kernel_size=3,stride=1,padding=1,bias=False)
        
        self.upsample1= nn.ConvTranspose2d(96,96,kernel_size=2,stride=2,padding=0);
        self.decode1_1 =nn.Conv2d(144,48,kernel_size=3,stride=1,padding=1,bias=False)
        self.decode1_2 =nn.Conv2d(48,48,kernel_size=3,stride=1,padding=1,bias=False)
        
        self.decode0=nn.Conv2d(48,1,kernel_size=3,stride=1,padding=1,bias=False)
        
    def forward(self,x):
        enc1 = torch.relu(self.encode1_1(x))
        enc1 = torch.relu(self.encode1_2(enc1))
        enc1 =self.bn1(enc1)
        
        enc2 = F.max_pool2d(enc1, kernel_size=2, stride=2, padding=0)
        enc2 = torch.relu(self.encode2_1(enc2))
        enc2 = torch.relu(self.encode2_2(enc2))
        enc2 =self.bn2(enc2)
        
        enc3 = F.max_pool2d(enc2, kernel_size=2, stride=2, padding=0)
        enc3 = torch.relu(self.encode3_1(enc3))
        enc3 = torch.relu(self.encode3_2(enc3))
        enc3 =self.bn3(enc3)
        
        enc4 = F.max_pool2d(enc3, kernel_size=2, stride=2, padding=0)       
        enc4 = torch.relu(self.encode4_1(enc4))
        enc4 = torch.relu(self.encode4_2(enc4))
        enc4 =self.bn3(enc4)
        
        #解码部分
        dec = self.upsample3(enc4)
        dec = torch.relu(self.decode3_1(torch.cat([enc3,dec],dim=1)))
        dec = torch.relu(self.decode3_2(dec))
        dec =self.bn3(dec)
        
        dec = self.upsample2(dec)
        dec = torch.relu(self.decode2_1(torch.cat([enc2,dec],dim=1)))
        dec = torch.relu(self.decode2_2(dec))
        dec =self.bn2(dec)
    
        dec = self.upsample1(dec)
        dec = torch.relu(self.decode1_1(torch.cat([enc1,dec],dim=1)))
        dec = torch.relu(self.decode1_2(dec))
        dec =self.bn1(dec)
        
        out = self.bn0(torch.tanh(self.decode0(dec)))
        return out


In [11]:
#torch.cuda.get_device_name(0)
#torch.cuda.device_count()
torch.cuda.is_available()

print(torch.cuda.is_available()) # true GPU avaiable

print(torch.cuda.device_count()) #GPU num， 1

torch.cuda.current_device() # index of GPU， 0

torch.cuda.get_device_name(0) #GPU name

True
1


'GeForce 940MX'

In [12]:
#########定义损失函数
class My_loss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,out_pred,mix,m_h):
        lossh = mix* (out_pred - m_h)           #[[B, C, H, W]]
#         lossl = mix* ((1-out_pred) - m_l)           #[[B, C, H, W]]
        loss=torch.sum(torch.sum(torch.pow(lossh, 2), 1))#
        return loss
    
criterion = My_loss()
# criterion = nn.MSELoss(reduction='sum')  # 用于计算MSE损失

In [13]:
## hyper parameter

lr = 0.01
model = MyUnet()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler  = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
optimizer.step()#
scheduler.step()

criterion.cuda(0)
model.cuda(0)

MyUnet(
  (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn0): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encode1_1): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode1_2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode2_1): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode2_2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode3_1): Conv2d(96, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode3_2): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encode4_1): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(

In [14]:
log_step=10#test model every 10 epchos
for epoch in range(200):
    start_time = time.time()
    model.train()# training
    mean_loss = 0.
    for index, (mix,m_h) in enumerate(train_loader,0): 
#         mix=mix.cuda(0);m_h=m_h.cuda(0)
        optimizer.zero_grad()
        out_pred= model(mix)
        loss = criterion(out_pred,mix,m_h)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        mean_loss += loss.data.item()
    mean_loss /= (index+1)
    print('-' * 70)
    print('|epoch{:4d}|time:{:5.2f}s|train_loss:{:5.2f}|'.format(epoch,(time.time()-start_time),mean_loss))
    if epoch %log_step == 0:
        torch.save(model.state_dict(),'MyUnet_0_epoch=%d.pt' % (epoch))
        model.eval()
        mean_loss=torch.tensor(0).float()
        for index, (mix,m_h) in enumerate(test_loader,0):
#             mix=mix.cuda(0);m_h=m_h.cuda(0)
            out_pred=model(mix)
            loss = criterion(out_pred,mix,m_h)
            
            mean_loss+= loss.data.item()
        mean_loss /= (index+1)      
        print('*' * 70)
        print('|epoch{:3d}|time:{:4.2f}s|test loss:{:5.2f}|'.format(epoch,(time.time()-start_time),mean_loss))
        print('*' * 70)     
            

----------------------------------------------------------------------
|epoch   0|time: 3.89s|train_loss:12452.30|
**********************************************************************
|epoch  0|time:4.71s|test loss:106997.30|
**********************************************************************
----------------------------------------------------------------------
|epoch   1|time: 3.86s|train_loss:7135.78|
----------------------------------------------------------------------
|epoch   2|time: 3.88s|train_loss:6076.03|


KeyboardInterrupt: 

In [ ]:
#保存模型
torch.save(model.state_dict(), 'my_unet0_鼾声.pt')

In [ ]:

#输入验证信号：
dataset=myDataset(mix="mix_20.csv",m_h="m_h_20.csv",
                  sig_num=20,seq_len=seq_len,input_size=input_size)
valid_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False,num_workers=0) 



In [ ]:

    
for index, (mix,m_h) in enumerate(valid_loader,0):
    if encuda:
        mix=mix.cuda();m_h=m_h.cuda();
    out_pred=model(mix)
    loss = criterion(out_pred,mix,m_h)
    print('第{:3d}个信号的loss: {:5.2f}'.format(index+1,loss))
    # -----------------输出预测的mask-------------
    out_h=out_pred;out_l=1-out_pred
    h_mask=torch.reshape(out_h,(256,128));l_mask=torch.reshape(out_l,(256,128))

    h_mask=h_mask.cpu();l_mask=l_mask.cpu()
    h_mask=h_mask.detach().numpy();l_mask=l_mask.detach().numpy()
    np.savetxt('h_mask%d.csv'%(index+1), h_mask, delimiter = ',')
    np.savetxt('l_mask%d.csv'%(index+1), l_mask, delimiter = ',')
#     print(h_mask.shape);print(l_mask.shape)
    # -----------------输出预测的mask-------------